In [2]:
from mpl_toolkits.mplot3d import axes3d, Axes3D
import pandas as pd
import lifelines
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from lifelines.datasets import load_larynx

In [3]:
%matplotlib inline
%matplotlib notebook


class Ploting:
    
    def __init__(self,vectors,variable,param_strata,param_gKM=None,angle=None,distribution=None):
        
        self.color_mapping={"viridis":plt.cm.viridis,"coolwarm":plt.cm.coolwarm\
                            ,"inferno":plt.cm.inferno,"plasma":plt.cm.plasma\
                            ,"cividis":plt.cm.cividis,"magma":plt.cm.magma}
        self.param=param_strata
        self.param_gKM=param_gKM
        self.X=vectors[0]
        self.Y=vectors[1]
        self.Z=vectors[2]
        if self.param_gKM!=None:
            self.gKM_vectors=vectors[3:]
        
        self.xlabel=variable
        self.ylabel="time"
        self.zlabel="probability or surviving the event untill t"
        
        self.angle=angle
        self.fig= None
        self.axe= None
        self.distribution=distribution
    
    def init_gKM(self):
        #change config
        self.X=self.gKM_vectors[0]
        self.Y=self.gKM_vectors[1]
        self.Z=self.gKM_vectors[2]
        self.param=self.param_gKM
        
        
        #clear
        
        self.gKm_vectors=None
        self.param_gKM=None
        
    def help_color(self):
        print("color : ", *tuple(self.color_mapping.keys()))
        
    def plot_survival_3D(self):
        self.set_figure()
        self.chose_mode()
        self.plot_distribution()
        return self.fig,self.axe
    
    def chose_mode(self):
        if self.param["shape"]=="trisurf":
            self.plot_trisurf()
        elif self.param["shape"]=="surface":
            self.plot_surface()
        elif self.param["shape"]=="wireframe":
            self.plot_wireframe()
        elif self.param["shape"]=="contour":
            self.plot_contour()
        if self.param_gKM!=None:
            self.init_gKM()
            self.chose_mode()
    
    def set_figure(self):
        self.fig=plt.figure(figsize=(8,8))
        self.axe = self.fig.add_subplot(1,1,1, projection='3d')
        self.axe.set_xlabel(self.xlabel,**config_axes)
        self.axe.set_ylabel(self.ylabel,**config_axes)
        self.axe.set_zlabel(self.zlabel,**config_axes)
        if self.angle!=None :
            assert(len(self.angle)==2)
            anglexz,anglexy=self.angle[0],self.angle[1]
            self.axe.view_init(anglexz,anglexy)
    
    def plot_distribution(self):
        if str(type(self.distribution))=="<class 'dict'>":
            self.axe.bar(self.distribution["distribution"].index, self.distribution["distribution"].values.T[0]*self.distribution["scaler"], \
                         zs=0, zdir='z', label='curve in (x,y)')
            
    def plot_surface(self):
        self.axe.plot_surface(self.X,self.Y,self.Z, **self.param["param"])
        
    def plot_wireframe(self):
        self.axe.plot_wireframe(self.X,self.Y,self.Z, **self.param["param"])
    
    def plot_trisurf(self):
        self.axe.plot_trisurf(self.X,self.Y,self.Z, **self.param["param"])
    
    def plot_contour(self):
        self.axe.contour(self.X,self.Y,self.Z, **self.param["param"] )
    
    
from lifelines import CoxPHFitter
class Curves_df:
    
    
    def __init__(self,dic_event=None):
        
        self.Fitter=None
        self.event=dic_event
        #self.interpolation
    
    def fit(self,df,variable,regular_bins=True,global_KMe=False):
        self.Fitter=CoxPHFitter().fit(df,"time","event",strata=[variable])
        strata=self.Fitter.baseline_survival_
        if global_KMe==False:
            return strata
        else :
            KMg=self.global_KMe(df,variable,regular_bins)
            return strata, KMg
        
    def global_KMe(self,df,variable,regular_bins=True):
        if regular_bins==True:
            n_copy=np.array(df[variable].unique())
            nmax=n_copy.max()
            nmin=n_copy.min()
            print(df.columns)
            self.Fitter=CoxPHFitter().fit(df[["time","event"]],"time","event")### Etrange .drop(columns=variable)
            X=self.Fitter.baseline_survival_
            X[nmax]=X.iloc[:,0]
            return X.rename(columns={X.columns[0]:nmin})
        
        
class Interpolation():
    pass

        
class Matrix_3D:
    def __init__(self,shape,dic_event=None):
        self.event=dic_event
        self.shape=shape
        
    def fit(self,df_pandas,vector_names,regular_bins=True):
        pass
    
    def transform(self,df_pandas,regular_bins=True):
        
        if self.shape in ["trisurf"]:
            """sample_coordinates"""
            if regular_bins==True:
                L=[]
                for i in range(len(df_pandas.columns)):
                    c=df_pandas.columns[i]
                    for j in df_pandas.index:
                        L+=[[c,j,df_pandas.loc[j,c]]]
            L=np.array(L).T            
            return L[0],L[1],L[2]              #pd.DataFrame(L,columns=vector_names)
        
        elif self.shape in ["surface","wireframe","contour"]:
            """mapping_2D_value"""
            X=df_pandas.columns.values.astype(float)
            Y=df_pandas.index.values.astype(float)
            Z=df_pandas.values.astype(float)
            x2, y2 = np.meshgrid(X, Y)
            return x2,y2,df_pandas
        
        
def TD_plot(D,variable,param_strata,param_gKM=None,angle=None,distribution=None):
    if param_gKM==None:
        C=Curves_df().fit(D,variable) 
        X,Y,Z=Matrix_3D(param_strata["shape"]).transform(C)
        return Ploting([X,Y,Z],variable,param_strata,angle=angle,distribution=distribution)\
              .plot_survival_3D()   
    else:
        Cv,Cg=Curves_df().fit(D,variable,global_KMe=True) 
        Xv,Yv,Zv,=Matrix_3D(param_strata["shape"]).transform(Cv)
        Xg,Yg,Zg=Matrix_3D(param_gKM["shape"]).transform(Cg)
        return Ploting([Xv,Yv,Zv,Xg,Yg,Zg],variable,param_strata,param_gKM,angle=angle,distribution=distribution)\
              .plot_survival_3D()   

In [ ]:
config_axes={"fontsize":"large","fontweight":"semibold","linespacing":10}

param_strata={"shape":"surface","param":{"cmap":"inferno","alpha":0.9,"linewidth":5}}
param_gKM={"shape":"surface","param":{"cmap":"viridis","alpha":0.4,"linewidth":1}}
dic_distribution={"distribution":distribution,"scaler":15}
TD_plot(D,"age",param_strata,param_gKM,distribution=dic_distribution)

In [4]:
df=pd.read_csv("sensor.csv")

EmptyDataError: No columns to parse from file